In [ ]:
%matplotlib notebook

dataset_location = "trash_detection.v2i.yolov5pytorch"
# or unzipped download from webpage: dataset_location = "trash-detection-2/"

## Read dataset information:

In [ ]:
# read number of classes based on YAML config file:
import yaml
with open(dataset_location + "/data.yaml", 'r') as stream:
    data_cfg = yaml.safe_load(stream)
    num_classes = str(data_cfg['nc'])
    list_classes = data_cfg['names']
    print("Dataset defines", num_classes, "classes ->\n", list_classes)
    

In [ ]:
from os import listdir
from os.path import isfile, join

train_folder=dataset_location + "/train"
test_folder=dataset_location + "/valid"

# read list of all image files in dataset folder
train_imgs = [f for f in listdir(train_folder+"/images") if (isfile(join(train_folder+"/images", f)) and f.endswith(".jpg"))]
test_imgs = [f for f in listdir(test_folder+"/images") if (isfile(join(test_folder+"/images", f)) and f.endswith(".jpg"))]


train_data=[]
for f in train_imgs:
    f_meta = join(train_folder+"/labels", f.replace(".jpg",".txt"))
    meta=[]
    if isfile(f_meta):
        with open(f_meta) as fc:
            lines = [line.rstrip() for line in fc]
            for line in lines:
                content = [float(value) for value in line.split(' ')]
                if len(content)==5:
                # classid, rect_x, rect_y rect_x2, rect_y2
                    content[0] = int(content[0])
                    meta.append(content)
        if len(meta)>0:
            train_data.append([join(train_folder+"/images", f), meta])

            
            
test_data=[]
for f in test_imgs:
    f_meta = join(test_folder+"/labels", f.replace(".jpg",".txt"))
    meta=[]
    if isfile(f_meta):
        with open(f_meta) as fc:
            lines = [line.rstrip() for line in fc]
            for line in lines:
                content = [float(value) for value in line.split(' ')]
                if len(content)==5:
                # classid, rect_x, rect_y rect_x2, rect_y2
                    content[0] = int(content[0])
                    meta.append(content)
        if len(meta)>0:
            test_data.append([join(train_folder+"/images", f), meta])
            
            
            
print("Num training roi boxes is", len(train_data))
print("Num testing roi boxes is", len(test_data))


In [ ]:
data_nr = 3
vis_data=train_data


import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

im = Image.open(vis_data[data_nr][0])
im_class_nr = [meta[0] for meta in vis_data[data_nr][1]]
im_class = [list_classes[clsmeta] for clsmeta in im_class_nr]
im_roi = [meta[1:] for meta in vis_data[data_nr][1]]

im_h = im.height
im_w = im.width

print("Dataset entry",data_nr, "dimension is", im_w, "x", im_h)
for clsname,roidata in zip(im_class, im_roi):
    print("ROI", clsname, "at pos", roidata)



# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(im)

# Create a Rectangle patch
rects=[]
for clsname,roidata in zip(im_class, im_roi):
    print("ROI", clsname, "at pos", roidata)
    roi_center = (im_w*roidata[0], im_h*roidata[1])
    roi_size = (im_w*roidata[2], im_h*roidata[3])
    roi_topleft = (roi_center[0]-roi_size[0]//2, roi_center[1]-roi_size[1]//2)
    rects.append( patches.Rectangle(roi_topleft, roi_size[0], roi_size[1], linewidth=1, edgecolor='r', facecolor='none'))
    

# Add the patch to the Axes
for rect in rects:
    ax.add_patch(rect)

plt.show()


In [ ]:

from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image



vis_data=train_data



fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)


def updateIm(nr = 1.0):
    print(vis_data[int(nr)][0])
    im = Image.open(vis_data[int(nr)][0])
    im_class_nr = [meta[0] for meta in vis_data[int(nr)][1]]
    im_class = [list_classes[clsmeta] for clsmeta in im_class_nr]
    im_roi = [meta[1:] for meta in vis_data[int(nr)][1]]

    im_h = im.height
    im_w = im.width
    
    ax.clear()
    # Display the image
    ax.imshow(im)

    # Create a Rectangle patch
    rects=[]
    for clsname,roidata in zip(im_class, im_roi):
        print("ROI", clsname, "at pos", roidata)
        roi_center = (im_w*roidata[0], im_h*roidata[1])
        roi_size = (im_w*roidata[2], im_h*roidata[3])
        roi_topleft = (roi_center[0]-roi_size[0]//2, roi_center[1]-roi_size[1]//2)
        rects.append( patches.Rectangle(roi_topleft, roi_size[0], roi_size[1], linewidth=1, edgecolor='r', facecolor='none'))

    # Add the patch to the Axes
    for rect in rects:
        ax.add_patch(rect)
    fig.canvas.draw_idle()

interact(updateIm, nr = (0,len(vis_data),1));

